# ACP Project - Systematic Model Comparison

In [1]:
import warnings, pickle, os, itertools
from dataclasses import dataclass
from joblib import Parallel, delayed, parallel_backend

try:
    from sklearnex import patch_sklearn
    patch_sklearn()
except ImportError:
    pass

import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 300)

from IPython.display import display
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(rc={'figure.figsize':(10,10)})

import shap
import optuna
optuna.logging.set_verbosity(optuna.logging.INFO)

%load_ext autoreload
%autoreload 1

In [2]:
class Notebook:
    CROSS_VAL_N_JOBS = 1
    N_PROCESSES = 1
    N_TRIALS = 1
    TRIALS_TIMEOUT = 60 * 60 * 2
    TRIAL_STORAGE = None  # "sqlite:///models/studies.db"
    RUN = False
    PERSIST_MODELS = True
    MODEL_DIR = "models/systematic"


if Notebook.PERSIST_MODELS:
    try:
        os.makedirs(Notebook.MODEL_DIR)
    except FileExistsError:
        pass


In [3]:
from sklearn.model_selection import train_test_split
from dataset import SCIData, SCICols
%aimport dataset

sci = SCIData.load('data/sci.h5')

scii = (
    SCIData(SCIData.quickload("data/sci_processed.h5").sort_values("AdmissionDateTime"))
    .mandate(SCICols.news_data_raw)
    .derive_critical_event(within=1, return_subcols=True)
    .augment_shmi(onehot=True)
    .omit_redundant()
    .raw_news()
    .derive_ae_diagnosis_stems(onehot=False)
    .categorize()
   # .onehot_encode_categories()
)

sci_train, sci_test, _, y_test_mortality, _, y_test_criticalcare = train_test_split(
    scii,
    scii.DiedWithinThreshold,
    scii.CriticalCare,
    test_size=0.33,
    random_state=42,
    shuffle=False,
)
sci_train, sci_test = SCIData(sci_train), SCIData(sci_test)
# (X_train, y_train), (X_test, y_test) = (
#     sci_train.xy(outcome="CriticalEvent", dropna=False, fillna=False),
#     sci_test.xy(outcome="CriticalEvent", dropna=False, fillna=False),
# )

In [4]:
from sklearn.base import BaseEstimator
from typing import Dict, Any, Iterable


class Estimator:
    _name: str
    _estimator: BaseEstimator
    _requirements: Dict[str, bool]
    _static_params: Dict[str, Any] = {}
    _tuning_params_default: Dict[str, Any] = {}
    _fit_params: Dict[str, Any] = {}

    def __init__(self, sci_train):
        pass

    @classmethod
    def suggest_parameters(cls, trial):
        return dict()

    @classmethod
    def compile_parameters(cls, params):
        return {
            f"{cls._name}__{key}": value
            for key, value in {
                **cls._static_params,
                **cls._tuning_params_default,
                **params,
            }.items()
        }

    @classmethod
    def factory(cls):
        return cls._estimator(**cls._static_params)

    @classmethod
    def fit_params(cls, X_train, y_train):
        return {
            f'{cls._name}__{key}': value
            for key, value in cls._fit_params.items()
        }


In [5]:
from imblearn.over_sampling import SMOTENC
from imblearn.under_sampling import RandomUnderSampler
from imblearn import FunctionSampler


class Resampler_SMOTE(Estimator):
    _name = "SMOTE"
    _estimator = SMOTENC

    _static_params = dict(random_state=42, n_jobs=None,)

    _tuning_params_default = dict(sampling_strategy=0.1, k_neighbors=5)

    @classmethod
    def suggest_parameters(cls, trial):
        suggestions = dict(
            sampling_strategy=trial.suggest_float(
                f"{cls._name}__sampling_strategy", 0.1, 0.5
            ),
            k_neighbors=trial.suggest_int(f"{cls._name}__k_neighbors", 2, 10),
        )

        return {
            f"{cls._name}__kw_args": {
                **cls._static_params,
                **cls._tuning_params_default,
                **suggestions,
            }
        }

    @classmethod
    def factory(cls):
        return FunctionSampler(
            func=SCIData.SMOTE, validate=False, kw_args=cls._static_params
        )


class Resampler_RandomUnderSampler(Estimator):
    _name = "RandomUnderSampler"
    _estimator = RandomUnderSampler

    _static_params = dict(random_state=42, replacement=False)

    _tuning_params_default = dict(sampling_strategy=0.1)

    @classmethod
    def suggest_parameters(cls, trial):
        return cls.compile_parameters(
            dict(
                sampling_strategy=trial.suggest_float(
                    f"{cls._name}__sampling_strategy", 0.05, 0.5
                )
            )
        )


class No_Resampling(Estimator):
    _name = "No_Resampling"

    @classmethod
    def suggest_parameters(cls, trial):
        return dict()

    @staticmethod
    def _(X, y):
        return X, y

    @classmethod
    def factory(cls):
        return FunctionSampler(func=cls._, validate=False)


In [6]:
from lightgbm import LGBMClassifier
from sklearn.calibration import CalibratedClassifierCV


class Estimator_LightGBM(Estimator):
    _name = "LightGBM"
    _estimator = LGBMClassifier

    _requirements = dict(onehot=False, ordinal=False, imputation=False, fillna=False, resampling=False)

    _static_params = dict(
        objective="binary",
        metric=["l2", "auc"],
        boosting_type="gbdt",
        n_jobs=1,
        random_state=42,
        verbose=-1,
    )

    _tuning_params_default = dict(
        is_unbalance=True,
        reg_alpha=1.8e-3,
        reg_lambda=6e-4,
        num_leaves=14,
        colsample_bytree=0.4,
        subsample=0.97,
        subsample_freq=1,
        min_child_samples=6,
    )

    @classmethod
    def suggest_parameters(cls, trial):
        suggestions = dict(
            reg_alpha=trial.suggest_float(
                f"{cls._name}__reg_alpha", 1e-4, 10.0, log=True
            ),
            reg_lambda=trial.suggest_float(
                f"{cls._name}__reg_lambda", 1e-4, 10.0, log=True
            ),
            num_leaves=trial.suggest_int(f"{cls._name}__num_leaves", 2, 256),
            colsample_bytree=trial.suggest_float(
                f"{cls._name}__colsample_bytree", 0.4, 1.0
            ),
            subsample=trial.suggest_float(f"{cls._name}__subsample", 0.4, 1.0),
            subsample_freq=trial.suggest_int(f"{cls._name}__subsample_freq", 1, 7),
            min_child_samples=trial.suggest_int(
                f"{cls._name}__min_child_samples", 5, 150
            ),
            is_unbalance=trial.suggest_categorical(
                f"{cls._name}__is_unbalance", [True, False]
            ),
        )

        if not suggestions["is_unbalance"]:
            suggestions["scale_pos_weight"] = trial.suggest_int(
                f"{cls._name}__scale_pos_weight", 1, 100
            )

        r = cls.compile_parameters(suggestions)
        if not suggestions["is_unbalance"]:
            del r[f"{cls._name}__is_unbalance"]

        return r


In [7]:
from xgboost import XGBClassifier


class Estimator_XGBoost(Estimator):
    _name = "XGBoost"
    _estimator = XGBClassifier

    _requirements = dict(onehot=False, ordinal=False, imputation=False, fillna=False, resampling=False)

    _static_params = dict(
        verbosity=0,
        n_jobs=1,
        objective="binary:logistic",
        booster="gbtree",
        enable_categorical=True,
    )

    _tuning_params_default = {
        **dict(
            tree_method="hist",
            alpha=7e-05,
            subsample=0.42,
            colsample_bytree=0.87,
            scale_pos_weight=14,
            max_depth=7,
            min_child_weight=10,
            eta=0.035,
            gamma=4e-08,
            grow_policy="lossguide",
        ),
        "lambda": 7e-2,
    }

    @classmethod
    def suggest_parameters(cls, trial):
        suggestions = dict(
            tree_method=trial.suggest_categorical(
                f"{cls._name}__tree_method", ["approx", "hist"]
            ),
            alpha=trial.suggest_float(f"{cls._name}__alpha", 1e-8, 1.0, log=True),
            subsample=trial.suggest_float(f"{cls._name}__subsample", 0.2, 1.0),
            colsample_bytree=trial.suggest_float(
                f"{cls._name}__colsample_bytree", 0.2, 1.0
            ),
            scale_pos_weight=trial.suggest_int(
                f"{cls._name}__scale_pos_weight", 1, 100
            ),
            max_depth=trial.suggest_int(f"{cls._name}__max_depth", 3, 9, step=2),
            min_child_weight=trial.suggest_int(f"{cls._name}__min_child_weight", 2, 10),
            eta=trial.suggest_float(f"{cls._name}__eta", 1e-8, 1.0, log=True),
            gamma=trial.suggest_float(f"{cls._name}__gamma", 1e-8, 1.0, log=True),
            grow_policy=trial.suggest_categorical(
                f"{cls._name}__grow_policy", ["depthwise", "lossguide"]
            ),
        )
        suggestions["lambda"] = trial.suggest_float(
            f"{cls._name}__lambda", 1e-8, 1.0, log=True
        )

        return cls.compile_parameters(suggestions)


In [8]:
from sklearn.linear_model import LogisticRegression


class Estimator_LogisticRegression(Estimator):
    _name = "LogisticRegression"
    _estimator = LogisticRegression

    _requirements = dict(onehot=True, ordinal=False, imputation=True, fillna=True, resampling=False)

    _static_params = dict(max_iter=100, solver="lbfgs", random_state=42, penalty="l2")

    _tuning_params_default = dict(penalty="l2", C=5.9, class_weight="balanced")

    @classmethod
    def suggest_parameters(cls, trial):
        suggestions = dict(
            penalty=trial.suggest_categorical(f"{cls._name}__penalty", ["l2", "none"]),
            C=trial.suggest_float(f"{cls._name}__C", 0.01, 10),
            class_weight=trial.suggest_categorical(
                f"{cls._name}__class_weight", [None, "balanced"]
            ),
        )

        # if suggestions["penalty"] == "elasticnet":
        #     suggestions["l1_ratio"] = trial.suggest_float(
        #         f"{cls._name}__l1_ratio", 0.05, 0.95
        #     )

        return cls.compile_parameters(suggestions)



In [9]:
from sklearn.ensemble import RandomForestClassifier
class Estimator_RandomForest(Estimator):
    _estimator = RandomForestClassifier
    _name = "RandomForest"

    _requirements = dict(onehot=False, ordinal=True, imputation=False, fillna=True, resampling=False)
    _tuning_params_default = dict(
        n_estimators = 250,
        max_features=0.56,
        min_samples_split=8,
        min_samples_leaf=3,
        max_samples=0.75,
        class_weight='balanced',
    )

    @classmethod
    def suggest_parameters(cls, trial):
        suggestions = dict(
            n_estimators = trial.suggest_int(
                f'{cls._name}__n_estimators', 25, 250
            ),
            max_features = trial.suggest_float(
                f'{cls._name}__max_features', 0.15, 1.0
            ),
            min_samples_split = trial.suggest_int(
                f'{cls._name}__min_samples_split', 2, 15
            ),
            min_samples_leaf = trial.suggest_int(
                f'{cls._name}__min_samples_leaf', 1, 15
            ),
            max_samples = trial.suggest_float(
                f'{cls._name}__max_samples', 0.5, 0.99
            ),
            class_weight = trial.suggest_categorical(
                f'{cls._name}__class_weight', [None, 'balanced', 'balanced_subsample']
            ),
        )

        return cls.compile_parameters(suggestions)

In [10]:
from utils.isolation_forest_wrapper import IsolationForestWrapper
class Estimator_IsolationForest(Estimator):
    _name = 'IsolationForest'
    _estimator = IsolationForestWrapper
    _requirements = dict(onehot=True, ordinal=False, imputation=True, fillna=True, resampling=False)

    _tuning_params_default = dict(
        n_estimators = 140,
        max_samples = 0.45,
        contamination = 0.02,
        max_features = 0.69,
        bootstrap=False
    )

    @classmethod
    def suggest_parameters(cls, trial):
        suggestions = dict(
            n_estimators = trial.suggest_int(
                f'{cls._name}__n_estimators', 1, 200
            ),
            max_samples = trial.suggest_float(
                f'{cls._name}__max_samples', 0.0, 1.0
            ),
            contamination = trial.suggest_float(
                f'{cls._name}__contamination', 1e-6, 1e-1
            ),
            max_features = trial.suggest_float(
                f'{cls._name}__max_features', 0.0, 1.0
            ),
            bootstrap = trial.suggest_categorical(
                f'{cls._name}__bootstrap', [True, False]
            ),
        )

        return cls.compile_parameters(suggestions)



In [23]:
import torch
from pytorch_tabnet.tab_model import TabNetClassifier

@dataclass
class TabNetWrapper(TabNetClassifier):
    weights: int = 0
    max_epochs: int = 100
    patience: int = 10
    batch_size: int = 1024
    virtual_batch_size: int = 128
    drop_last: bool = True
    eval_metric: str = None

    def fit(self, X, y):
        return super().fit(X_train=X.to_numpy(), y_train=y.to_numpy(), eval_metric = self.eval_metric, weights=self.weights, max_epochs=self.max_epochs, patience=self.patience, batch_size=self.batch_size, virtual_batch_size=self.virtual_batch_size, drop_last=self.drop_last)

    def predict(self, X):
        return super().predict(X.to_numpy())
    
    def predict_proba(self, X):
        return super().predict_proba(X.to_numpy())

    def decision_function(self, X):
        return self.predict_proba(X)[:, 1]

In [12]:
class Estimator_TabNet(Estimator):
    _estimator = TabNetWrapper
    _name = 'TabNet'
    _requirements = dict(onehot=False, ordinal=True, imputation=True, fillna=True, resampling=False)

    _static_params = dict(
        optimizer_fn = torch.optim.Adam,
        scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,
        verbose = 0,
        device_name="cuda" if torch.cuda.is_available() else "cpu",
        scheduler_params = dict(
            mode = 'min',
            min_lr = 1e-5,
            factor = 0.5
        ),
        optimizer_params = dict(
            lr = 2e-2,
            weight_decay=1e-5
        ),
        cat_emb_dim = 1,
        max_epochs = 50,
        eval_metric='average_precision',
        weights=1,
        drop_last=False
    )

    _tuning_params_default = dict(
        n_d = 8,
        n_a = 8,
        n_steps = 3,
        gamma = 1.2,
        lambda_sparse = 8e-4,
        mask_type='sparsemax',
        n_shared=3,
        scheduler_params=dict(
            patience=5
        ),
    )

    def __init__(self, sci_train):
        self._categorical_idxs, self._categorical_dims = sci_train.describe_categories(dimensions=True)
    
    def factory(self):
        return self._estimator(
            cat_idxs = self._categorical_idxs,
            cat_dims = [_+1 for _ in self._categorical_dims], # Because we may add 1 category when we fill_na
            **self._static_params
        )

    @classmethod
    def suggest_parameters(cls, trial):
        suggestions = dict(
            n_steps = trial.suggest_int(
                f'{cls._name}__n_steps', 1, 10
            ),
            n_shared = trial.suggest_int(
                f'{cls._name}__n_shared', 1, 10
            ),
            gamma = trial.suggest_float(
                f'{cls._name}__gamma', 1, 1.5
            ),
            lambda_sparse = trial.suggest_float(
                f'{cls._name}__lambda_sparse', 1e-6, 1e-3, log=True
            ),
            mask_type = trial.suggest_categorical(
                f'{cls._name}__mask_type', ['entmax', 'sparsemax']
            ),
            scheduler_params = dict(
                patience = trial.suggest_int(
                    f'{cls._name}__scheduler__patience', 3, 10
                )
            )
        )

        n_da = trial.suggest_int(
            f'{cls._name}__n_da', 4, 32, 
        )
        suggestions['n_d'], suggestions['n_a'] = n_da, n_da

        return cls.compile_parameters(suggestions)

    @classmethod
    def compile_parameters(cls, params):
        r = {
            **cls._static_params,
            **cls._tuning_params_default,
            **params,
            'scheduler_params': {
                **cls._static_params['scheduler_params'],
                **params['scheduler_params']
            }
        }
        return {
            f"{cls._name}__{key}": value
            for key, value in r.items()
        }


In [13]:
def get_feature_studies(sci_train):
    news = SCICols.news_data_raw
    news_extended = SCICols.news_data_raw + SCICols.news_data_extras
    labs = SCICols.blood
    hospital = [
        "AdmissionMethodDescription",
        "AdmissionSpecialty",
        "SentToSDEC",
        "Readmission",
    ]
    ae = ["AandEPresentingComplaint", "AandEMainDiagnosis"]
    diagnoses = [_ for _ in sci_train.columns if _.startswith("SHMI__")]
    phenotype = ["Female", "Age"]

    return list(
        dict(
            news=news,
            news_extended=news_extended,
            news_with_phenotype=news_extended + phenotype,
            with_ae_notes=news_extended + phenotype + ae,
            with_labs=news_extended + phenotype + labs,
            with_notes_and_labs=news_extended + phenotype + ae + labs,
            with_hospital=news_extended + phenotype + hospital,
            with_notes_and_hospital=news_extended + phenotype + ae + hospital,
            with_labs_and_hospital=news_extended + phenotype + labs + hospital,
            with_labs_and_diagnoses=news_extended + phenotype + labs + diagnoses,
            all=news_extended + phenotype + ae + labs + hospital + diagnoses,
        ).items()
    )


def get_studies(sci_train, study_grid = None):
    if study_grid is None:
        study_grid = dict(
            estimator=[Estimator_LogisticRegression, Estimator_LightGBM, Estimator_XGBoost],
            resampler=[None, Resampler_RandomUnderSampler, Resampler_SMOTE],
            features=get_feature_studies(sci_train),
        )

    k, v = zip(*study_grid.items())
    return [dict(zip(k, _)) for _ in itertools.product(*v)]


In [14]:
from imblearn.pipeline import Pipeline as ImbPipeline
from typing import Optional


class Pipeline(ImbPipeline):
    def persist(self, filename):
        with open(filename, "wb") as file:
            pickle.dump(self, file)

    @classmethod
    def load(cls, filename):
        with open(filename, "rb") as file:
            return pickle.load(file)


class PipelineFactory:
    def __init__(
        self,
        estimator: Estimator,
        X_train,
        y_train,
        resampler: Optional[Estimator] = None,
    ):
        (self._estimator, self._resampler, self._X_train, self._y_train,) = (
            estimator,
            resampler,
            X_train,
            y_train,
        )

    def __call__(self, **kwargs):
        steps = [
            (self._estimator._name, self._estimator.factory(),),
        ]
        if self._resampler is not None:
            steps = [
                (
                    self._resampler._name,
                    self._resampler.factory(),
                ),
            ] + steps

        return Pipeline(steps=steps).set_params(**kwargs)


In [15]:
from sklearn.model_selection import cross_validate

class Objective:
    def __init__(
        self,
        estimator: Estimator,
        resampler: Estimator,
        X_train,
        y_train,
        cv=5,
        scoring="average_precision",
        persist_model=True,
    ):
        (
            self._estimator,
            self._resampler,
            self._X_train,
            self._y_train,
            self._cv,
            self._scoring,
            self._persist_model,
        ) = (estimator, resampler, X_train, y_train, cv, scoring, persist_model)

        self._best_score = 0
        self._best_model = None

        self._pipeline_factory = PipelineFactory(
            estimator=self._estimator,
            resampler=self._resampler,
            X_train=self._X_train,
            y_train=self._y_train,
        )

        self._fit_params = self._estimator.fit_params(self._X_train, self._y_train)

    def __call__(self, trial):
        trial_params = {
            **(self._resampler.suggest_parameters(trial) if self._resampler else {}),
            **self._estimator.suggest_parameters(trial),
        }
        model = self._pipeline_factory(**trial_params)

        score = cross_validate(
            model,
            self._X_train,
            self._y_train,
            cv=self._cv,
            scoring=self._scoring,
            n_jobs=Notebook.CROSS_VAL_N_JOBS,
            fit_params=self._fit_params
        )["test_score"].mean()

        if self._persist_model and (score > self._best_score):
            self._best_score = score
            self._best_model = self._pipeline_factory(**trial_params).fit(
                self._X_train, self._y_train
            )

        return score


In [16]:
from typing import Dict, Any, Iterable, Optional, Tuple


def construct_study(
    estimator: Estimator,
    sci_train: SCIData,
    sci_test: SCIData,
    features: Tuple[str, Iterable[str]] = ("All", []),
    resampler: Estimator = None,
    cv=5,
    scoring="average_precision",
    storage=Notebook.TRIAL_STORAGE,
):
    X_train, y_train = sci_train.xy(
        x=features[1],
        imputation=estimator._requirements["imputation"],
        onehot_encoding=estimator._requirements["onehot"],
        ordinal_encoding=estimator._requirements['ordinal'],
        fillna=estimator._requirements["fillna"],
    )

    name = f"{estimator._name}_{resampler._name if resampler else 'None'}_{features[0]}"
    objective = Objective(
        estimator=estimator(SCIData(sci_train[features[1]])),
        resampler=resampler(SCIData(sci_train[features[1]])),
        X_train=X_train,
        y_train=y_train,
        cv=cv,
        scoring=scoring,
        persist_model=Notebook.PERSIST_MODELS,
    )
    study = optuna.create_study(direction="maximize", study_name=name, storage=storage)

    def call(**kwargs):
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore")
            study.optimize(objective, **kwargs)

        if Notebook.PERSIST_MODELS:
            objective._best_model.persist(f"{Notebook.MODEL_DIR}/{name}")

        return objective._best_model

    return call


In [17]:
optuna.logging.set_verbosity(optuna.logging.WARNING)
study_grid = dict(
    #estimator=[Estimator_LightGBM, Estimator_IsolationForest, Estimator_LogisticRegression, Estimator_RandomForest, Estimator_TabNet, Estimator_XGBoost],
    estimator=[Estimator_TabNet],
    resampler=[No_Resampling],
    features=get_feature_studies(sci_train),
)

for _ in get_studies(sci_train, study_grid)[5:]:
    s = construct_study(
        **_, sci_train=SCIData(sci_train.head(1000)), sci_test=sci_test
    )

    r = s(n_trials=2)


No early stopping will be performed, last training weights will be used.
No early stopping will be performed, last training weights will be used.
No early stopping will be performed, last training weights will be used.
No early stopping will be performed, last training weights will be used.
No early stopping will be performed, last training weights will be used.
No early stopping will be performed, last training weights will be used.
No early stopping will be performed, last training weights will be used.
No early stopping will be performed, last training weights will be used.
No early stopping will be performed, last training weights will be used.
No early stopping will be performed, last training weights will be used.
No early stopping will be performed, last training weights will be used.
No early stopping will be performed, last training weights will be used.
No early stopping will be performed, last training weights will be used.
No early stopping will be performed, last training 

[W 2022-11-01 21:40:36,007] Trial 0 failed because of the following error: KeyboardInterrupt()
Traceback (most recent call last):
  File "c:\Users\stybl\miniconda3\envs\py39\lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "C:\Users\stybl\AppData\Local\Temp\ipykernel_2412\2572392098.py", line 43, in __call__
    score = cross_validate(
  File "c:\Users\stybl\miniconda3\envs\py39\lib\site-packages\sklearn\model_selection\_validation.py", line 266, in cross_validate
    results = parallel(
  File "c:\Users\stybl\miniconda3\envs\py39\lib\site-packages\joblib\parallel.py", line 1046, in __call__
    while self.dispatch_one_batch(iterator):
  File "c:\Users\stybl\miniconda3\envs\py39\lib\site-packages\joblib\parallel.py", line 861, in dispatch_one_batch
    self._dispatch(tasks)
  File "c:\Users\stybl\miniconda3\envs\py39\lib\site-packages\joblib\parallel.py", line 779, in _dispatch
    job = self._backend.apply_async(batch, call

KeyboardInterrupt: 

In [ ]:
if Notebook.RUN:
    studies = [
        construct_study(**_, sci_train=sci_train, sci_test=sci_test)
        for _ in get_studies(sci_train)
    ]

    print("Starting execution..")
    with parallel_backend("loky", inner_max_num_threads=Notebook.CROSS_VAL_N_JOBS):
        results = Parallel(n_jobs=Notebook.N_PROCESSES)(
            delayed(_)(n_trials=Notebook.N_TRIALS, timeout=Notebook.TRIALS_TIMEOUT)
            for _ in studies
        )
